In [ ]:
# import librarys
import os
import cv2
import numpy as np
import glob as glob
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.applications.vgg16 import VGG16
from keras.models import Sequential
from keras.models import Model
from keras.layers import Input, Dense, Flatten

In [ ]:
# read image's class length
path = "./image/"
folders = os.listdir(path)
classes = [f for f in folders if os.path.isdir(os.path.join(path, f))]
n_classes = len(classes)
n_classes

In [ ]:
# create empty list (X = images, Y = class label)
X = []
Y = []

In [ ]:
# append data
for label, class_name in enumerate(classes):
    files = glob.glob(path + "/" + class_name + "/*.jpg")
    for file in files:
        img = cv2.imread(file)
        print(os.path.basename(file))
        img = cv2.resize(img, dsize=(224, 224))
        X.append(img)
        Y.append(label)

In [ ]:
X = np.array(X)
X = X.astype('float32')
X /= 255.0

In [ ]:
Y = np.array(Y)
Y = to_categorical(Y, n_classes)
print(Y)

In [ ]:
# split dataset to train and test data (train = 0.8, test = 0.2)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
print(X_train.shape)
print(Y_train.shape)

In [ ]:
# setup VGG16 model
input_tensor = Input(shape=(224,224,3))
base_model = VGG16(weights='imagenet', input_tensor=input_tensor, include_top=False)

top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(n_classes, activation='softmax'))

model = Model(inputs=base_model.input, outputs=top_model(base_model.output))

for layer in model.layers[:15]:
    layer.trainable = False
    
print('# layers=', len(model.layers))

model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

model.summary()

In [ ]:
# fit train data to model
model.fit(X_train, Y_train, epochs=50, batch_size=16)
# evaluate model use to test data
score = model.evaluate(X_test, Y_test, batch_size=16)

In [ ]:
# save model use to pickle
import pickle
pickle.dump(classes, open('backend/models/classes.sav', 'wb'))
model.save('backend/models/cnn.h5')